# Financial Data Extraction **bold text**

> This code shows you the process of extracting data and view it a collab setting.



In [9]:
pip install notebook

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.7 MB/s eta 0:00:00


In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('/content/sec_financial extractions.xlsx', sheet_name='Companies')
print(df.head())

     Company  Year  Revenue  Net Income  Total Assets  Total Liabilty  \
0      Apple  2024    93736       93736        364980           48390   
1      Apple  2023    96773       96995        352583           43009   
2      Apple  2022    81462       99803             0               0   
3  Microsoft  2024   245122       88136        512163               0   
4  Microsoft  2023   211915       72361        411976               0   

   Cash Flow from Operation  
0                    118254  
1                    110543  
2                    122151  
3                         0  
4                         0  


In [8]:
df.columns = df.columns.str.strip()

In [3]:
df = df.sort_values(['Company', 'Year'])

In [4]:
# Fill NA values that result from pct_change calculations with 0 or an appropriate value
df.fillna(0, inplace=True)

In [5]:
df['Revenue Growth (%)'] = df.groupby(['Company'])['Revenue'].pct_change() * 100
df['Net Income Growth (%)'] = df.groupby(['Company'])['Net Income'].pct_change() * 100


In [6]:
# Display the dataframe to verify the calculations
print(df)

     Company  Year  Revenue  Net Income  Total Assets  Total Liabilty  \
2      Apple  2022    81462       99803             0               0   
1      Apple  2023    96773       96995        352583           43009   
0      Apple  2024    93736       93736        364980           48390   
5  Microsoft  2022   198270       72738             0               0   
4  Microsoft  2023   211915       72361        411976               0   
3  Microsoft  2024   245122       88136        512163               0   
8      Tesla  2022    81462       12587             0               0   
7      Tesla  2023    96773       14974        106618           43009   
6      Tesla  2024    97690        7153        122070           48390   

   Cash Flow from Operation  Revenue Growth (%)  Net Income Growth (%)  
2                    122151                 NaN                    NaN  
1                    110543           18.795267              -2.813543  
0                    118254           -3.138272   

In [7]:
summary = df.groupby('Company').agg({
    'Revenue Growth (%)': 'mean',
    'Net Income Growth (%)': 'mean'
}).reset_index()

print("\nYear-over-Year Average Growth Rates (%):")
print(summary)


Year-over-Year Average Growth Rates (%):
     Company  Revenue Growth (%)  Net Income Growth (%)
0      Apple            7.828497              -3.086755
1  Microsoft           11.275996              10.641059
2      Tesla            9.871422             -16.633261


# Sales Prediction for **2025**

> We will need to add more libraries to our code to get the right format.

In [10]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [21]:
!pip install scikit-learn
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [11]:
# Method 1: Simple Average Growth Rate
def predict_with_avg_growth(df):
    """Predict using average historical growth rate"""
    predictions = []

    for company in df['Company'].unique():
        company_data = df[df['Company'] == company].sort_values('Year')

        if len(company_data) >= 2:
            # Calculate average growth rate
            revenues = company_data['Revenue'].values
            growth_rates = []

            for i in range(1, len(revenues)):
                if revenues[i-1] != 0:
                    growth_rate = (revenues[i] - revenues[i-1]) / revenues[i-1]
                    growth_rates.append(growth_rate)

            avg_growth = np.mean(growth_rates) if growth_rates else 0
            latest_revenue = revenues[-1]
            predicted_2025 = latest_revenue * (1 + avg_growth)

            predictions.append({
                'Company': company,
                'Method': 'Average Growth',
                'Latest_Revenue_2024': latest_revenue,
                'Avg_Growth_Rate': avg_growth * 100,
                'Predicted_2025': predicted_2025
            })

    return pd.DataFrame(predictions)

In [12]:
def predict_with_linear_regression(df):
    """Predict using linear regression on year vs revenue"""
    predictions = []

    for company in df['Company'].unique():
        company_data = df[df['Company'] == company].sort_values('Year')

        if len(company_data) >= 2:
            X = company_data['Year'].values.reshape(-1, 1)
            y = company_data['Revenue'].values

            # Fit linear regression
            model = LinearRegression()
            model.fit(X, y)

            # Predict 2025
            predicted_2025 = model.predict([[2025]])[0]
            r2 = r2_score(y, model.predict(X))

            predictions.append({
                'Company': company,
                'Method': 'Linear Regression',
                'R2_Score': r2,
                'Predicted_2025': predicted_2025,
                'Trend_Slope': model.coef_[0]
            })

    return pd.DataFrame(predictions)

In [17]:
def predict_with_polynomial_regression(df, degree=2):
    """Predict using polynomial regression"""
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score

    predictions = []

    for company in df['Company'].unique():
        company_data = df[df['Company'] == company].sort_values('Year')

        if len(company_data) >= 3:  # Need more points for polynomial
            X = company_data['Year'].values.reshape(-1, 1)
            y = company_data['Revenue'].values

            # Create polynomial features
            poly_features = PolynomialFeatures(degree=degree)
            X_poly = poly_features.fit_transform(X)

            # Fit polynomial regression
            model = LinearRegression()
            model.fit(X_poly, y)

            # Predict 2025
            X_2025_poly = poly_features.transform([[2025]])
            predicted_2025 = model.predict(X_2025_poly)[0]
            r2 = r2_score(y, model.predict(X_poly))

            predictions.append({
                'Company': company,
                'Method': f'Polynomial (degree {degree})',
                'R2_Score': r2,
                'Predicted_2025': predicted_2025
            })

    return pd.DataFrame(predictions)

In [18]:
def predict_with_weighted_average(df):
    """Predict using weighted moving average"""
    predictions = []

    for company in df['Company'].unique():
        company_data = df[df['Company'] == company].sort_values('Year')

        if len(company_data) >= 2:
            revenues = company_data['Revenue'].values
            years = company_data['Year'].values

            # Create weights (more recent years get higher weights)
            weights = np.arange(1, len(revenues) + 1)
            weights = weights / weights.sum()

            # Calculate weighted average growth
            growth_rates = []
            for i in range(1, len(revenues)):
                if revenues[i-1] != 0:
                    growth_rate = (revenues[i] - revenues[i-1]) / revenues[i-1]
                    growth_rates.append(growth_rate)

            if growth_rates:
                # Weight the growth rates
                growth_weights = weights[1:]  # Skip first weight since we have n-1 growth rates
                growth_weights = growth_weights / growth_weights.sum()
                weighted_growth = np.average(growth_rates, weights=growth_weights)
            else:
                weighted_growth = 0

            latest_revenue = revenues[-1]
            predicted_2025 = latest_revenue * (1 + weighted_growth)

            predictions.append({
                'Company': company,
                'Method': 'Weighted Average',
                'Latest_Revenue_2024': latest_revenue,
                'Weighted_Growth_Rate': weighted_growth * 100,
                'Predicted_2025': predicted_2025
            })

    return pd.DataFrame(predictions)

In [19]:
def create_ensemble_prediction(df):
    """Combine multiple prediction methods"""

    # Get predictions from different methods
    avg_growth_pred = predict_with_avg_growth(df)
    linear_pred = predict_with_linear_regression(df)
    poly_pred = predict_with_polynomial_regression(df)
    weighted_pred = predict_with_weighted_average(df)

    ensemble_predictions = []

    for company in df['Company'].unique():
        predictions = []
        weights = []

        # Collect predictions and their weights (based on method reliability)
        for pred_df, weight in [(avg_growth_pred, 0.25), (linear_pred, 0.30),
                               (poly_pred, 0.20), (weighted_pred, 0.25)]:
            company_pred = pred_df[pred_df['Company'] == company]
            if not company_pred.empty:
                predictions.append(company_pred['Predicted_2025'].iloc[0])
                weights.append(weight)

        if predictions:
            # Normalize weights
            weights = np.array(weights) / np.sum(weights)
            ensemble_pred = np.average(predictions, weights=weights)

            ensemble_predictions.append({
                'Company': company,
                'Method': 'Ensemble',
                'Predicted_2025': ensemble_pred,
                'Individual_Predictions': predictions,
                'Method_Weights': weights.tolist()
            })

    return pd.DataFrame(ensemble_predictions)


In [20]:
def visualize_predictions(df, predictions_df):
    """Create visualizations of historical data and predictions"""

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    companies = df['Company'].unique()

    for i, company in enumerate(companies):
        if i < 3:  # Only plot first 3 companies
            company_data = df[df['Company'] == company].sort_values('Year')

            # Plot historical data
            axes[i].plot(company_data['Year'], company_data['Revenue'],
                        'o-', label='Historical', linewidth=2, markersize=8)

            # Plot predictions
            pred_2025 = predictions_df[predictions_df['Company'] == company]['Predicted_2025'].iloc[0]
            axes[i].plot([2024, 2025], [company_data['Revenue'].iloc[-1], pred_2025],
                        'r--o', label='Prediction', linewidth=2, markersize=8)

            axes[i].set_title(f'{company} Revenue Trend & 2025 Prediction')
            axes[i].set_xlabel('Year')
            axes[i].set_ylabel('Revenue (millions)')
            axes[i].legend()
            axes[i].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [22]:
if __name__ == "__main__":
    # Load your data
    # df = pd.read_excel('/content/sec_financial extractions.xlsx', sheet_name='Companies')

    # Run all prediction methods
    print("=== 2025 Sales Predictions ===\n")

    # Method 1: Average Growth
    avg_predictions = predict_with_avg_growth(df)
    print("1. Average Growth Method:")
    print(avg_predictions.round(2))
    print()

    # Method 2: Linear Regression
    linear_predictions = predict_with_linear_regression(df)
    print("2. Linear Regression Method:")
    print(linear_predictions.round(2))
    print()

    # Method 3: Polynomial Regression
    poly_predictions = predict_with_polynomial_regression(df)
    print("3. Polynomial Regression Method:")
    print(poly_predictions.round(2))
    print()

    # Method 4: Weighted Average
    weighted_predictions = predict_with_weighted_average(df)
    print("4. Weighted Moving Average Method:")
    print(weighted_predictions.round(2))
    print()

    # Method 5: Ensemble
    ensemble_predictions = create_ensemble_prediction(df)
    print("5. Ensemble Method (Recommended):")
    print(ensemble_predictions.round(2))

    # Create summary comparison
    summary_data = []
    for company in df['Company'].unique():
        row = {'Company': company}

        # Get predictions from each method
        if not avg_predictions[avg_predictions['Company'] == company].empty:
            row['Avg_Growth'] = avg_predictions[avg_predictions['Company'] == company]['Predicted_2025'].iloc[0]
        if not linear_predictions[linear_predictions['Company'] == company].empty:
            row['Linear_Reg'] = linear_predictions[linear_predictions['Company'] == company]['Predicted_2025'].iloc[0]
        if not poly_predictions[poly_predictions['Company'] == company].empty:
            row['Polynomial'] = poly_predictions[poly_predictions['Company'] == company]['Predicted_2025'].iloc[0]
        if not weighted_predictions[weighted_predictions['Company'] == company].empty:
            row['Weighted_Avg'] = weighted_predictions[weighted_predictions['Company'] == company]['Predicted_2025'].iloc[0]
        if not ensemble_predictions[ensemble_predictions['Company'] == company].empty:
            row['Ensemble'] = ensemble_predictions[ensemble_predictions['Company'] == company]['Predicted_2025'].iloc[0]

        summary_data.append(row)

    summary_df = pd.DataFrame(summary_data)
    print("\n=== Summary: All Methods Comparison ===")
    print(summary_df.round(0))


=== 2025 Sales Predictions ===

1. Average Growth Method:
     Company          Method  Latest_Revenue_2024  Avg_Growth_Rate  \
0      Apple  Average Growth                93736             7.83   
1  Microsoft  Average Growth               245122            11.28   
2      Tesla  Average Growth                97690             9.87   

   Predicted_2025  
0       101074.12  
1       272761.95  
2       107333.39  

2. Linear Regression Method:
     Company             Method  R2_Score  Predicted_2025  Trend_Slope
0      Apple  Linear Regression      0.57       102931.00       6137.0
1  Microsoft  Linear Regression      0.95       265287.67      23426.0
2      Tesla  Linear Regression      0.79       108203.00       8114.0

3. Polynomial Regression Method:
     Company                 Method  R2_Score  Predicted_2025
0      Apple  Polynomial (degree 2)       1.0         72351.0
1  Microsoft  Polynomial (degree 2)       1.0        297891.0
2      Tesla  Polynomial (degree 2)       1.0  